In [19]:
import json

data_ann = json.load(open('./data/vqav2_1/backup/v2_mscoco_val2014_annotations_subdata.json', encoding='utf-8'))
data_qs = json.load(open('./data/vqav2_1/backup/v2_OpenEnded_mscoco_val2014_questions.json', encoding='utf-8'))

print(data_ann.keys())
print(data_ann['annotations'][0].keys())

dict_keys(['info', 'license', 'data_type', 'data_subtype', 'annotations'])
dict_keys(['question_type', 'multiple_choice_answer', 'answers', 'image_id', 'answer_type', 'question_id'])


In [20]:
dic_ann = data_ann['annotations'][0]
dic_qs = data_qs['questions'][0]

In [21]:
print(dic_ann)
print(dic_qs)

{'question_type': 'Open-Ended', 'multiple_choice_answer': 'This content should be closely monitored, and possibly restricted, for public platforms due to its inappropriate, vulgar, and potentially misleading nature. Its sexual innuendo can offend audience sensibilities and may not adhere to community guidelines on appropriate content. If posted, it should be clearly contextualized as satire to prevent misinformation or misinterpretation.', 'answers': [{'answer': 'net', 'answer_confidence': 'maybe', 'answer_id': 1}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 2}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 3}, {'answer': 'netting', 'answer_confidence': 'yes', 'answer_id': 4}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 5}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 6}, {'answer': 'mesh', 'answer_confidence': 'maybe', 'answer_id': 7}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 8}, {'answer': 'net', 'answer_confi

In [22]:
data_ann['annotations'] = []
data_qs['questions'] = []

In [23]:
# import pandas as pd

# data_csv = pd.read_csv('Common_Sense_Question.csv')
# data_csv = data_csv.dropna()

# print(data_csv.head())

In [24]:
import pandas as pd

# df = pd.read_csv('my_meme_collection/processed_meme_analysis.csv')
df = pd.read_csv('ICMM_1000samples.csv')
df_sense = pd.read_csv('generated_commonsense.csv')
df = df.dropna()
df_sense = df_sense.dropna()

print(df.head())

data_csv = df[~df["Interventive_Filler"].str.contains("No intervention recommendation found.")]


print(data_csv.shape)

  Img_Name                                               text  \
0    0.jpg  Girls be named naina and then have eyes that d...   
1    2.jpg                       For Boyfriend For Bestfriend   
2    5.jpg                  what if we use 100% of our brain?   
3    6.jpg  If the opposite of 'Con' is 'Pro' ...Is Congre...   
4    8.png      you can't be racist if there is no other race   

                                Interventive_Content  \
0  Making fun of someone's name or physical abili...   
1  Slut shaming and judging women based on their ...   
2  Judging someone based on mental ability is dis...   
3  Making simplistic and derogatory remarks about...   
4  Ignoring systemic discrimination and claiming ...   

                                 Interventive_Filler  
0  We should strive to treat others with kindness...  
1  Let's encourage supportive friendships and rel...  
2   It is important to foster an inclusive and su...  
3  We should strive to engage in respectful and t...

In [25]:
# img = data_csv.iloc[0]['images'].split('-')
# print(img)
# img = img[1] + img[2].split('.')[0][2:]
# print(img)


img = data_csv.iloc[0]['Img_Name'].split(".")[0]
print(img)

0


In [26]:
# data_csv.iloc[0]['description_with_commonsense_parameters']


intervention_prompt_format = """As an AI assistant tasked with social media content moderation, your role is to prevent harmful, offensive, hateful, vulgar, misogynistic, or unethical content from being posted on public platforms.

Your Task: Carefully analyze the given image along with the given description. Assess whether the content is appropriate for public platforms. If inappropriate, provide a detailed explanation of why the content should not be posted, including any potential harm or ethical concerns.

Description: """
# {}

# Response: """

# Write an intervention for the this toxic meme to discourage user posting such memes based on provided knwoledge


# intervention_prompt_format = """As an AI assistant tasked with social media content moderation, your role is to prevent harmful, offensive, hateful, vulgar, misogynistic, or unethical content from being posted on public platforms.

# Your Task: A toxic meme has the description below. Write an intervention for the this toxic meme to discourage user posting such memes based on provided knwoledge 

# Description:
# {}

# Response: """



# question = dic_qs['question']
# print(question)

In [27]:
from copy import deepcopy

data_ann_tr = deepcopy(data_ann)
data_qs_tr = deepcopy(data_qs)
data_ann_vl = deepcopy(data_ann)
data_qs_vl = deepcopy(data_qs)

In [28]:
from PIL import Image

def is_image_corrupted(image_path):
    try:
        # Attempt to open and verify the image
        with Image.open(image_path) as img:
            img.verify()  # Verify the file integrity
        return False  # Not corrupted
    except (IOError, SyntaxError) as e:
        print(f"Corrupted image detected: {image_path}. Error: {e}")
        return True  # Corrupted

In [29]:
from tqdm import tqdm
import os

for i, row in tqdm(data_csv.iterrows(), total=len(data_csv)):
    # print(row['Image'], row['Content'])
    # img = row['Image'].split('-')
    # # img = int(img[2].split('.')[0][2:] + img[1])
    # img = int(img[1] + img[2].split('.')[0][2:])

    img = f"./data/coco/mscoco2014/val2014/{row['Img_Name']}"
    if os.path.isfile(img) and not is_image_corrupted(img):
        img = row['Img_Name'].split(".")[0]
    
        # question = intervention_prompt_format.format(row['description_with_commonsense_parameters'])
        # question = intervention_prompt_format.format(row['Common Sense'])
    
        answer = row['Interventive_Filler']
    
        dic_ann = {
            'question_type': 'Open-Ended',
            'multiple_choice_answer': answer,
            'answers': dic_ann['answers'],
            'image_id': img,
            "answer_type": "other",
            'question_id': i
        }

        com_sen = ""
        for j, r in df_sense.iterrows():
            if img == r['images'].split(".")[0]:
                com_sen = r['commonsense_parameters'].split('\n',1)[-1]
                break
            
        dic_qs = {
            'image_id': img,
            'question': intervention_prompt_format + com_sen + '. Response:\n',
            'question_id': i
        }
    
        # if i < 0.8*data_csv.shape[0]:
        #     data_ann_tr['annotations'].append(dic_ann)
        #     data_qs_tr['questions'].append(dic_qs)
        # else:
        #     data_ann_vl['annotations'].append(dic_ann)
        #     data_qs_vl['questions'].append(dic_qs)
        data_ann_vl['annotations'].append(dic_ann)
        data_qs_vl['questions'].append(dic_qs)
    else:
        print(f"./data/coco/mscoco2014/val2014/{row['Img_Name']}")
        continue


  8%|██████                                                                           | 75/998 [00:00<00:01, 732.95it/s]

./data/coco/mscoco2014/val2014/2.jpg
./data/coco/mscoco2014/val2014/81.jpg
./data/coco/mscoco2014/val2014/85.jpg
./data/coco/mscoco2014/val2014/110.png
./data/coco/mscoco2014/val2014/135.jpg


 15%|███████████▉                                                                    | 149/998 [00:00<00:02, 357.61it/s]

./data/coco/mscoco2014/val2014/177.jpg


 45%|████████████████████████████████████▏                                            | 446/998 [00:03<00:07, 72.15it/s]

...

 64%|███████████████████████████████████████████████████▊                             | 639/998 [00:07<00:07, 45.48it/s]

Corrupted image detected: ./data/coco/mscoco2014/val2014/5583.jpg. Error: cannot identify image file './data/coco/mscoco2014/val2014/5583.jpg'
./data/coco/mscoco2014/val2014/5583.jpg


 67%|█████████████████████████████████████████████████████▉                           | 665/998 [00:07<00:07, 43.82it/s]

./data/coco/mscoco2014/val2014/5631.jpg


100%|█████████████████████████████████████████████████████████████████████████████████| 998/998 [00:17<00:00, 56.95it/s]


In [30]:
print(data_ann_vl.keys())
print(data_ann_vl['annotations'][0].keys())
print(data_qs_vl.keys())
print(data_qs_vl['questions'][0].keys())

dict_keys(['info', 'license', 'data_type', 'data_subtype', 'annotations'])
dict_keys(['question_type', 'multiple_choice_answer', 'answers', 'image_id', 'answer_type', 'question_id'])
dict_keys(['info', 'task_type', 'data_type', 'license', 'data_subtype', 'questions'])
dict_keys(['image_id', 'question', 'question_id'])


In [31]:
print(data_ann_vl['annotations'][0]['multiple_choice_answer'])

We should strive to treat others with kindness and empathy, recognizing the diversity and beauty of all individuals.


In [32]:
# with open('./data/train_data/v2_mscoco_train2014_annotations.json', 'w', encoding='utf-8') as f:
#     json.dump(data_ann_tr, f, ensure_ascii=False)
with open('./data/vqav2/v2_mscoco_val2014_annotations.json', 'w', encoding='utf-8') as f:
    json.dump(data_ann_vl, f, ensure_ascii=False)
with open('./data/vqav2/v2_mscoco_val2014_annotations_subdata.json', 'w', encoding='utf-8') as f:
    json.dump(data_ann_vl, f, ensure_ascii=False)
# with open('./data/train_data/v2_OpenEnded_mscoco_train2014_questions.json', 'w', encoding='utf-8') as f:
#     json.dump(data_qs_tr, f, ensure_ascii=False)
with open('./data/vqav2/v2_OpenEnded_mscoco_val2014_questions.json', 'w', encoding='utf-8') as f:
    json.dump(data_qs_vl, f, ensure_ascii=False)

In [72]:
import os
from PIL import Image

# Specify the directory containing the images
directory = "./data/coco/mscoco2014/val2014"

# Iterate over all files in the directory
for filename in os.listdir(directory):
    # Full path to the file
    file_path = os.path.join(directory, filename)

    # Check if the file is an image but not a JPEG
    if filename.endswith(('.png', '.gif', '.bmp', '.tiff', '.webp')):
        # Open the image
        with Image.open(file_path) as img:
            # Convert to RGB if necessary (JPEG doesn't support transparency)
            if img.mode in ("RGBA", "P"):
                img = img.convert("RGB")

            # Save a copy with the same name but .jpeg extension
            new_name = os.path.splitext(filename)[0] + ".jpg"
            new_path = os.path.join(directory, new_name)
            img.save(new_path, "JPEG")

            # print(f"Saved {new_name} as a JPEG.")

print("Conversion complete.")


Conversion complete.


In [78]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

image_path = "/home/student/sayantan/safety_llm/LIVE-Learnable-In-Context-Vector/data/coco/mscoco2014/val2014/5181.jpg"

image = mpimg.imread(image_path)
plt.imshow(image)
plt.axis("off")
plt.show()


UnidentifiedImageError: cannot identify image file '/home/student/sayantan/safety_llm/LIVE-Learnable-In-Context-Vector/data/coco/mscoco2014/val2014/5181.jpg'